In [140]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel

import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [15]:
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model

In [42]:
# Load data
houses = pd.read_csv('houses.csv', index_col=0)
houses_competition = pd.read_csv('houses_competition.csv', index_col=0)
houses_competition.insert(0, 'Fake price', np.nan) # to have same indexes for predictions

In [43]:
X = houses.iloc[:,1:]
y = houses.iloc[:,0]
y = y.values
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [44]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [52]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(100, activation='relu')(input_layer)
dense_layer_2 = Dense(64, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(64, activation='relu')(dense_layer_2)
dense_layer_4 = Dense(25, activation='relu')(dense_layer_3)
output = Dense(1)(dense_layer_4)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 18)]              0         
_________________________________________________________________
dense_29 (Dense)             (None, 100)               1900      
_________________________________________________________________
dense_30 (Dense)             (None, 64)                6464      
_________________________________________________________________
dense_31 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_32 (Dense)             (None, 25)                1625      
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 26        
Total params: 14,175
Trainable params: 14,175
Non-trainable params: 0
_________________________________________________

In [53]:
history = model.fit(X_train, y_train, batch_size=2, epochs=100, verbose=1, validation_split=0.2)

Epoch 1/100
4287/4287 [==============================] - 18s 4ms/step - loss: 4.0213 - mean_squared_error: 4.0213 - val_loss: 38.0158 - val_mean_squared_error: 38.0158
Epoch 2/100
4287/4287 [==============================] - 20s 5ms/step - loss: 2.8319 - mean_squared_error: 2.8319 - val_loss: 2.7245 - val_mean_squared_error: 2.7245
Epoch 3/100
4287/4287 [==============================] - 19s 4ms/step - loss: 2.7913 - mean_squared_error: 2.7913 - val_loss: 2.3467 - val_mean_squared_error: 2.3467
Epoch 4/100
4287/4287 [==============================] - 16s 4ms/step - loss: 2.2132 - mean_squared_error: 2.2132 - val_loss: 2.2097 - val_mean_squared_error: 2.2097
Epoch 5/100
4287/4287 [==============================] - 15s 4ms/step - loss: 2.1343 - mean_squared_error: 2.1343 - val_loss: 2.1550 - val_mean_squared_error: 2.1550
Epoch 6/100
4287/4287 [==============================] - 21s 5ms/step - loss: 2.0819 - mean_squared_error: 2.0819 - val_loss: 2.2808 - val_mean_squared_error: 2.2808
Ep

In [54]:
pred = model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, pred)))

1.2961536202987098


In [75]:
X_competition = houses_competition.iloc[:,1:]
Y_competition = houses_competition.iloc[:,0]

X_competition = sc.transform(X_competition)

pred = model.predict(X_competition)
pred = pd.DataFrame({'ID': Y_competition.index, 'Price': pred.flatten()})
pred.to_csv('predictions/submission.csv', index=False)

In [74]:
pred = pd.DataFrame({'Price': pred.flatten()})

In [78]:
X = houses.iloc[:,1:]
Y = houses.iloc[:,0]
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [110]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", 
                                random_state=20, 
                                learning_rate=0.1,
                                n_estimators=1000,
                                max_depth=5,
                                subsample=0.8,
                                colsample_bytree=0.8,
                                gamma=10)
xgb_model.fit(X_train, y_train)

# MSE
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

clear_output()
mse

1.217070092922464

In [112]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", 
                                random_state=20, 
                                learning_rate=0.1,
                                n_estimators=1000,
                                max_depth=5,
                                subsample=0.8,
                                colsample_bytree=0.8,
                                gamma=5)
xgb_model.fit(X_train, y_train)

# MSE
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

clear_output()
mse

1.1786110540503665

In [158]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", 
                                random_state=20, 
                                learning_rate=0.1,
                                n_estimators=1000,
                                max_depth=5,
                                subsample=0.8,
                                colsample_bytree=0.8,
                                gamma=5)
xgb_model.fit(X_train, y_train)

# MSE
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

clear_output()
mse

1.1786110540503665

In [157]:
X_competition = houses_competition.iloc[:,1:]
Y_competition = houses_competition.iloc[:,0]

pred = xgb_model.predict(X_competition)

pred = pd.DataFrame({'ID': Y_competition.index, 'Price': pred})
pred.to_csv('predictions/submission.csv', index=False)

/home/henri/anaconda3/envs/tfx-last/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
